# Calibration of motor current

This script takes the measurements of the 08-01-2022 test at the rotorcraft test facility to calibrate voltage and current measurements for the firefly vehicle 

## Import of necessary python modules and definition of motor class

In [ ]:
%matplotlib notebook
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# custom moving average function that can handle numpy arrays as well as dictionaries containing numpy arrays
from tools.data_loader import moving_average

import glob
import os

In [ ]:
class Motor:
    """ Object to store voltage and current data for each individual motor """
    
    def __init__(self, motorNumber, escFilepath, multimeterFilepath):
        
        # save numer of motor (1 to 8)
        self.motorNumber = motorNumber
        
        # filepaths for esc and multimeter data
        self.escFilepath = escFilepath
        self.multimeterFilepath = multimeterFilepath
        
        # data from ESCs
        self.currentESC = self.get_ESC_current(motorNumber)
        self.voltageESC = self.get_ESC_voltage(motorNumber)
        
        # multimeter measurements
        
        voltageDataMultimeter = self.get_multimeter_voltage(motorNumber)
        currentDataMultimeter = self.get_multimeter_current(motorNumber)
        
        self.voltageMultimeter = voltageDataMultimeter.iloc[motorNumber-1].values                
        self.currentMultimeter = currentDataMultimeter.iloc[0].values
        
        # power setting for each measured data point 
        self.throttleSetpointVoltage = np.array([int(v[1:]) for v in voltageDataMultimeter.columns])
        self.throttleSetpointCurrent = np.array([int(v[1:]) for v in currentDataMultimeter.columns])
        
        
        # timescale, RPM and voltage during voltage measurement
        
        self.timeVoltage = ((self.voltageESC['t'].values - min(self.voltageESC['t'].values)) / 10e5)
        self.timeCurrent = ((self.currentESC['t'].values - min(self.currentESC['t'].values)) / 10e5)
            
        self.omegaVoltage = self.voltageESC[f'omega{motorNumber}'].values
        self.omegaCurrent = self.currentESC[f'omega{motorNumber}'].values
        
        self.voltage = self.voltageESC[f'U1{motorNumber}'].values
        self.current = self.currentESC[f'I1{motorNumber}'].values      
               
        # apply filter functions filter out the measurement intervalls
        self.voltageSequences = self.get_voltage_sequence()
        self.currentSequences = self.get_current_sequence()
        
        # perform regression for RPM
        self.throttleRange = np.linspace(20, 100, 100).reshape(-1,1)
        self.omegaRegressionCurve = self.rpm_regression()
        self.ESCcurrentRegressionCurve = self.esc_current_regression()
        self.multimeterCurrentRegressionCurve = self.multimeter_current_regression()
        
        # print out voltage and current bias
        print(f'Motor {self.motorNumber}: voltageBias = {np.around(self.voltageBias,decimals=4)} V  ', end="")
        print(f'Motor {self.motorNumber}: currentBias = {np.around(self.currentBias,decimals=4)} A')
       
    
    @staticmethod
    def get_files_for_limit(limit, flight, discard=True):
        """ This function reads in the .csv-files for a given filter velocity threshold """
        
        # get filepath of .csv-files for a given limit
        data_directory = flight + f'hover_{limit}_limit'
        
        # get an (unsorted) list of .csv-files that represent one sequence
        csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
        

        # sort sequences in ascending order
        sorted_files = []
        for i in range(len(csv_files)):
            for e in csv_files:
                if f'sequence_{i}_' in e:
                    #print(e)
                    sorted_files.append(e)
                    break

        # discard first and land sequence (to remove data points from takeoff and after landing)
        if discard:
            return sorted_files[1:-1]

        else:
            return sorted_files
    
    
    def get_ESC_current(self, motorNumber):
        """ Extracts current data from the ESC data """
        
        filepath = self.escFilepath + f'current_motor_{motorNumber}/' 
        
        filesCurrent = self.get_files_for_limit(10, filepath, discard=False)
        
        currentData = pd.DataFrame()
        
        for file in filesCurrent:
            currentData = pd.concat([currentData, pd.read_csv(file)])
        
        return currentData
    
    
    
    def get_ESC_voltage(self, motorNumber):
        """ Extracts voltage data from the ESC data """
        
        filepath = self.escFilepath + f'/voltage_motor_{motorNumber}/'
        
        filesVoltage = self.get_files_for_limit(10, filepath, discard=False)
        
        voltageData = pd.DataFrame()
        
        for file in filesVoltage:
            voltageData = pd.concat([voltageData, pd.read_csv(file)])
        
        return voltageData
    
    
    def get_multimeter_voltage(self, motorNumber):
        """ Get multimeter voltage measurements from .csv-file """
        
        filepath = self.multimeterFilepath + 'voltage_data.csv'
        voltage_measurements = pd.read_csv(filepath)
        
        return voltage_measurements

    
    def get_multimeter_current(self, motorNumber):
        """ Get multimeter current measurements from .csv-file """
        
        filepath = self.multimeterFilepath + f'current_data_multimeter/current_motor{motorNumber}.csv'
        current_measurement = pd.read_csv(filepath)
        
        return current_measurement#.iloc[0].values
    
    
    def get_current_sequence(self):
        """ This function filters out the data points for the time during which the motor was running """
        
        # get all unique RPM values above 1000 RPM (this filters out time during which the motor was not spinning)
        # and the respective count
        relevantRpmValues = np.unique(self.omegaCurrent[self.omegaCurrent>=1000], return_counts=True) 
        
        # resolution of the RPM measurement is not very high (~ 30 RPM), so the values during the measurements
        # occur much more frequently than the values during acceleration and deceleration
        # all values that do not occur more than 20 times are discarded
        
        frequentRpmValues = relevantRpmValues[0][relevantRpmValues[1] > 20]
        
        # get filtered indices for RPM vales from current measurement
        filteredIndices = np.nonzero(np.isin(self.omegaCurrent, frequentRpmValues)==1)[0]
        
        # measurements form a coherent sequence of data points - in this step start and end points of the 
        # sequences are filtered by searching for gaps that are larger than 10 data points
        
        startMask = np.where(np.diff(filteredIndices,prepend=0)>10) # search for the gaps between the sequences
        endMask = np.where(np.abs(np.diff(np.flip(filteredIndices),prepend=0))>10)
        
        sequenceEnd = np.flip(np.flip(filteredIndices)[endMask])
        sequenceStart = filteredIndices[startMask]
        
        filteredCurrentSequence = []
        
        for i in range(sequenceStart.size):
            
            # extract data points that belong to the individual sequence
            intersection = np.intersect1d(np.arange(sequenceStart[i],sequenceEnd[i]), filteredIndices)
            
            # remove first and last 5 values to get rid of outliers
            if not intersection.size == 0:
                filteredCurrentSequence.append(intersection[5:-5])
            
        return filteredCurrentSequence
    
    
    def get_voltage_sequence(self):
        """ This function filters out the data points for the time during which the motor was running """
        
        # get all unique RPM values above 1000 RPM (this filters out time during which the motor was not spinning)
        # and the respective count
        relevantRpmValues = np.unique(self.omegaVoltage[self.omegaVoltage>=1000], return_counts=True) 
        
        # resolution of the RPM measurement is not very high (~ 30 RPM), so the values during the measurements
        # occur much more frequently than the values during acceleration and deceleration
        # all values that do not occur more than 20 times are discarded
        frequentRpmValues = relevantRpmValues[0][relevantRpmValues[1] > 20]
        
        # get filtered indices for RPM vales from current measurement
        filteredIndices = np.nonzero(np.isin(self.omegaVoltage, frequentRpmValues)==1)[0]
        
        # measurements form a coherent sequence of data points - in this step start and end points of the 
        # sequences are filtered by searching for gaps that are larger than 10 data points
        
        startMask = np.where(np.diff(filteredIndices,prepend=0)>10) # search for the gaps between the sequences
        
        endMask = np.where(np.abs(np.diff(np.flip(filteredIndices),prepend=0))>10)
        
        sequenceStart = filteredIndices[startMask]
        sequenceEnd = np.flip(np.flip(filteredIndices)[endMask])
        
        filteredVoltageSequence = []
        
        for i in range(sequenceStart.size):
            
            # extract data points that belong to the individual sequence
            intersection = np.intersect1d(np.arange(sequenceStart[i],sequenceEnd[i]), filteredIndices)
            
            # remove first and last 5 values to get rid of outliers
            if not intersection.size == 0:
                filteredVoltageSequence.append(intersection[5:-5])
            
        return filteredVoltageSequence
    
    
    def rpm_regression(self):
        """ Performs polynomial (quadratic) regression for RPM over throttle"""
        
        # initialize arrays for commanded throttle, mean omega and mean voltage for every sequence
        meanOmega = np.zeros(len(self.voltageSequences))
        meanVoltage = np.zeros(len(self.voltageSequences))
        
        # iterate through every voltage sequence to calcualte mean values for omega and voltage
        for i, seq in enumerate(self.voltageSequences):
                        
            omega = self.omegaVoltage[seq]
            voltage = self.voltage[seq]
            
            meanOmega[i] = np.mean(omega)
            meanVoltage[i] = np.mean(voltage)
        
        # calculate voltage bias by taking the mean difference between ESC-voltage and multimeter voltage
        self.voltageBias = np.sum(meanVoltage[:4] - self.voltageMultimeter[:4])/4
        
        # perform quadratic regression for the mean RPM values
        features = PolynomialFeatures(degree=2).fit_transform(self.throttleSetpointVoltage.reshape(-1,1))
        omegaRegression = LinearRegression().fit(features, meanOmega)
        omegaCurve = omegaRegression.predict(PolynomialFeatures(degree=2).fit_transform(self.throttleRange))
    
        return omegaCurve
    
    def esc_current_regression(self):
        """ Performs polynomial (quadratic) regression for RPM over throttle"""
                
        # initialize arrays for commanded throttle, mean omega and mean voltage for every sequence
        meanCurrent = np.zeros(len(self.currentSequences))
        
        # iterate through every voltage sequence to calcualte mean values for omega and voltage
        for i, seq in enumerate(self.currentSequences[:4]):
            
            current =  self.current[seq]
            meanCurrent[i] = np.mean(current)
        
        self.currentBias = np.sum(meanCurrent[:4] - self.currentMultimeter[:4])/4
        
        features = PolynomialFeatures(degree=2).fit_transform(self.throttleSetpointCurrent[:4].reshape(-1,1))
        currentRegression = LinearRegression().fit(features, meanCurrent[:4])
        currentCurve = currentRegression.predict(PolynomialFeatures(degree=2).fit_transform(self.throttleRange))
    
        return currentCurve
    
    def multimeter_current_regression(self):
            
        throttleSetpoints = self.throttleSetpointCurrent[:4].reshape(-1,1)
        
        features = PolynomialFeatures(degree=2).fit_transform(self.throttleSetpointCurrent[:4].reshape(-1,1))
        currentRegression = LinearRegression().fit(features, self.currentMultimeter[:4])
        currentCurve = currentRegression.predict(PolynomialFeatures(degree=2).fit_transform(self.throttleRange))
    
        return currentCurve
             

The following cell creates a motor object for all 8 motors and evaluates the current and voltage measurements that were taken during the ground test on 08-08-2022

In [ ]:
# insert filePath for esc and multimeter data
escFilepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/'
multimeterFilepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/'

# create motor object for all 8 motors and store them into a list

motor = []
for i in range(1,9):
    motor.append(Motor(i, escFilepath, multimeterFilepath))

    
# order in which the current and voltage was measured for each motor
motorOrderVoltage = [1, 6, 5, 2, 8, 4, 7, 3]
motorOrderCurrent = [1, 2, 5, 6, 4, 8, 3, 7]

# manufacturers motor data https://www.kdedirect.com/products/kde6213xf-185

# throttle setting in percentage [%]
manufacturerThrottle = np.array([0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1])*100

# RPM and current for each throttle setting
manufacturerRPM = np.array([1680, 2280, 2820, 3480, 3900, 4380, 4800])
manufacturerCurrent = np.array([1.8, 3.9, 7.7, 13.7, 19.7, 29.2, 38.8])

## perform linear regression for manufacturer data 

# power setting input from 20% to 100%
throttle = np.linspace(20, 100, 100).reshape(-1,1)

# quadratic regression for rpm
rpmFeatures = PolynomialFeatures(degree=2).fit_transform(manufacturerThrottle.reshape(-1,1))    
rpmRegression = LinearRegression().fit(rpmFeatures, manufacturerRPM)
manufacturerRPMCurve = rpmRegression.predict(PolynomialFeatures(degree=2).fit_transform(throttle))

# cubic regression for current
currentFeatures = PolynomialFeatures(degree=2).fit_transform(manufacturerThrottle.reshape(-1,1))
currentRegression = LinearRegression().fit(currentFeatures, manufacturerCurrent)
manufacturerCurrentCurve = currentRegression.predict(PolynomialFeatures(degree=2).fit_transform(throttle))


In [ ]:
# Show voltage measurements over time for all 8 motors

fig0, ax0 = plt.subplots(8,2, figsize=(10,32))
fig0.suptitle('Voltage measurements', y=0.9925, fontsize=16)

# raw data
for i in range(8):
    
    m = motorOrderVoltage[i]-1 # plots motors in the order of the measurements
    
    # plot RPM and voltage measurements over time (all data points)
    ax0[i][0].scatter(motor[m].timeVoltage, motor[m].omegaVoltage, color='lightgray', label='RPM raw')
    ax0[i][1].scatter(motor[m].timeVoltage, motor[m].voltage, color='lightgray', label='esc-voltage raw')

    # iterate over the filtered voltage sequences 
    for j, seq in enumerate(motor[m].voltageSequences):
        
        # get subset for time, omega, voltage for each sequence
        time = motor[m].timeVoltage[seq]
        omega =  motor[m].omegaVoltage[seq]
        voltage = motor[m].voltage[seq]
        
        # mean voltage value and midpoint of time intervall
        t = (max(time) + min(time))/2
        mean_voltage = np.mean(voltage)
        
        # plot sequences, plot legend entries for j=0
        if(j==0):
            ax0[i][0].scatter(time, omega, color='deepskyblue', label = 'RPM filtered')
            ax0[i][1].scatter(time, voltage, color='deepskyblue', label='esc-voltage filtered')
            ax0[i][1].scatter(t, mean_voltage, s=50, marker='x', color='b', label='esc-voltage mean')
            ax0[i][1].scatter(t, motor[m].voltageMultimeter[j], s=50, marker='x', color='r',label='multimeter-voltage')
        else:
            ax0[i][0].scatter(time, omega, color='deepskyblue')
            ax0[i][1].scatter(time, voltage, color='deepskyblue')
            ax0[i][1].scatter(t, mean_voltage, s=50, marker='x', color='b')
            ax0[i][1].scatter(t, motor[m].voltageMultimeter[j], s=50, marker='x', color='r')
    
    # set axis labels, limits, grid and legend for RPM plot
    ax0[i][0].set_xlabel('Time [s]')
    ax0[i][0].set_ylabel('$RPM$')
    ax0[i][0].set_ylim(0,5000)
    ax0[i][0].grid()
    ax0[i][0].legend()
    ax0[i][0].set_title(f'Motor {m+1}: RPM over time')
    
    # set axis labels, limits, grid and legend for voltage plot
    ax0[i][1].set_xlabel('Time [s]')
    ax0[i][1].set_ylabel('Voltage')
    ax0[i][1].set_ylim(29.5,33.5)
    ax0[i][1].grid()
    ax0[i][1].legend()
    ax0[i][1].set_title(f'Motor {m+1}: Voltage over time')

fig0.tight_layout()

In [ ]:
# Show current measurements for all motors

fig1, ax1 = plt.subplots(8, 2, figsize=(10,32))
fig1.suptitle('Current measurements', y=0.9925, fontsize=16)

# raw data
for i in range(8):
    
    m = motorOrderCurrent[i]-1 # plots motors in the order of the measurements
    
    # plot RPM and voltage measurements over time (all data points)
    ax1[i][0].scatter(motor[m].timeCurrent, motor[m].omegaCurrent, color='lightgray', label='RPM raw')
    ax1[i][1].scatter(motor[m].timeCurrent, motor[m].current, color='lightgray', label='esc-voltage raw')

    # iterate over the filtered voltage sequences 
    for j, seq in enumerate(motor[m].currentSequences):
        
        # get subset for time, omega, voltage for each sequence
        time = motor[m].timeCurrent[seq]
        omega =  motor[m].omegaCurrent[seq]
        current = motor[m].current[seq]
        
        # mean voltage value and midpoint of time intervall
        t = (max(time) + min(time))/2
        mean_current = np.mean(current)
        
        # plot sequences, plot legend entries for j=0
        if(j==0):
            ax1[i][0].scatter(time, omega, color='deepskyblue', label = 'RPM filtered')
            ax1[i][1].scatter(time, current, color='deepskyblue', label='esc-current filtered')
            ax1[i][1].scatter(t, mean_current, s=50, marker='x', color='b', label='esc-current mean')
            ax1[i][1].scatter(t, motor[m].currentMultimeter[j],
                              s=50, marker='x', color='r',label='multimeter-current')
        else:
            ax1[i][0].scatter(time, omega, color='deepskyblue')
            ax1[i][1].scatter(time, current, color='deepskyblue')
            ax1[i][1].scatter(t, mean_current, s=50, marker='x', color='b')
            ax1[i][1].scatter(t, motor[m].currentMultimeter[j], s=50, marker='x', color='r')
    
    # set axis labels, limits, grid and legend for RPM plot
    ax1[i][0].set_xlabel('Time [s]')
    ax1[i][0].set_ylabel('$RPM$')
    ax1[i][0].set_ylim(0,5000)
    ax1[i][0].grid()
    ax1[i][0].legend()
    ax1[i][0].set_title(f'Motor {m+1}: RPM over time')
    
    # set axis labels, limits, grid and legend for current plot
    ax1[i][1].set_xlabel('Time [s]')
    ax1[i][1].set_ylabel('Current')
    ax1[i][1].set_ylim(0,30)
    ax1[i][1].grid()
    ax1[i][1].legend()
    ax1[i][1].set_title(f'Motor {m+1}: Current over time')

fig1.tight_layout()

In [ ]:
# Plot regression curves for RPM and compare ESC and multimeter voltage

fig0, ax0 = plt.subplots(8,2, figsize=(10,32))
fig0.suptitle('Voltage measurements', y=0.9925, fontsize=16)

# raw data
for i in range(8):
    
    m = motorOrderVoltage[i]-1 # plots motors in the order of the measurements

    # plot RPM sequences and voltage measurements     
    for j, seq in enumerate(motor[m].voltageSequences):

        # get values for time (x-axis) and RPM/Voltage (y-axis)
        time = motor[m].timeVoltage[seq]
        omega =  motor[m].omegaVoltage[seq]
        voltage = motor[m].voltage[seq]
        
        # get values for the percentage of max. power that was set during the calibration test
        powerSetting = motor[m].throttleSetpointVoltage[j]
        powerSettingVector = np.ones(omega.size)* powerSetting
        
        # calculate mean voltage and RPM for every filtered sequence
        mean_voltage = np.mean(voltage)
        mean_omega = np.mean(omega)
        
        # plot sequences, plot legend entries for j=0
        if(j==0):
            ax0[i][0].scatter(powerSettingVector, omega, color='deepskyblue')
            ax0[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b', label='RPM measured')
            
            ax0[i][1].scatter(powerSettingVector, voltage, color='deepskyblue', label='esc-voltage filtered')
            ax0[i][1].scatter(powerSetting, mean_voltage, s=50, marker='x', color='b', label='esc-voltage mean')
            ax0[i][1].scatter(powerSetting, motor[m].voltageMultimeter[j], s=50,
                              marker='x', color='r',label='multimeter-voltage')
        else:
            ax0[i][0].scatter(powerSettingVector, omega, color='deepskyblue')
            ax0[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b')
            
            ax0[i][1].scatter(powerSettingVector, voltage, color='deepskyblue')
            ax0[i][1].scatter(powerSetting, mean_voltage, s=50, marker='x', color='b')
            ax0[i][1].scatter(powerSetting, motor[m].voltageMultimeter[j], s=50, marker='x', color='r')
    
    # manufacturer RPM data points + regression curve
    ax0[i][0].scatter(manufacturerThrottle, manufacturerRPM, color='black', marker='x', label='RPM manufacturer')
    ax0[i][0].plot(throttle, manufacturerRPMCurve, color='black', linewidth=1, linestyle='dashed')#, label='rpm regression')
    
    # regression curve for motor RPM
    ax0[i][0].plot(motor[m].throttleRange, motor[m].omegaRegressionCurve,
                   color='deepskyblue', linestyle='dashed', linewidth=1)

    # labels, title, grid, legend
    ax0[i][0].set_xlabel('Power [%]')
    ax0[i][0].set_ylabel('$RPM$')
    ax0[i][0].set_ylim(0,5000)
    ax0[i][0].grid()
    ax0[i][0].legend()
    ax0[i][0].set_title(f'Motor {m+1}: RPM over time')
    
    ax0[i][1].set_xlabel('Power [%]')
    ax0[i][1].set_ylabel('Voltage')
    ax0[i][1].set_ylim(25,35)
    ax0[i][1].grid()
    ax0[i][1].legend()
    ax0[i][1].set_title(f'Motor {m+1}: Voltage over time')
    
fig0.tight_layout()

In [ ]:
# Show current measurements for all motors

fig1, ax1 = plt.subplots(8, 2, figsize=(10,32))
fig1.suptitle('Current measurements', y=0.9925, fontsize=16)

# raw data
for i in range(8):
    
    m = motorOrderVoltage[i]-1 # plots motors in the order of the measurements

    # plot the sequences 

    for j, seq in enumerate(motor[m].currentSequences):
        
        # get values for time (x-axis) and RPM/Current (y-axis)
        time = motor[m].timeCurrent[seq]
        omega =  motor[m].omegaCurrent[seq]
        current = motor[m].current[seq]
        
        # get values for the percentage of max. power that was set during the calibration test
        powerSetting = motor[m].throttleSetpointCurrent[j]
        powerSettingVector = np.ones(omega.size)* powerSetting
        
        # calculate mean voltage and RPM for every filtered sequence
        mean_current = np.mean(current)
        mean_omega = np.mean(omega)
        
        # plot sequences, plot legend entries for j=0
        if(j==0):
            ax1[i][0].scatter(powerSettingVector, omega, color='deepskyblue', label = 'RPM filtered')
            ax1[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b', label='RPM measured')
            ax1[i][1].scatter(powerSettingVector, current, color='deepskyblue')#, label='esc-voltage filtered')
            ax1[i][1].scatter(powerSetting, mean_current, s=50, marker='x', color='b', label='esc-current mean')
            ax1[i][1].scatter(powerSetting, motor[m].currentMultimeter[j], s=50, marker='x', color='r',label='multimeter-current')
        
        else:
            ax1[i][0].scatter(powerSettingVector, omega, color='deepskyblue')
            ax1[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b')
            ax1[i][1].scatter(powerSettingVector, current, color='deepskyblue')
            ax1[i][1].scatter(powerSetting, mean_current, s=50, marker='x', color='b')
            ax1[i][1].scatter(powerSetting, motor[m].currentMultimeter[j], s=50, marker='x', color='r')
    
    # regression curve for motor RPM
    ax1[i][0].plot(motor[m].throttleRange, motor[m].omegaRegressionCurve,
                   color='deepskyblue', linestyle='dashed', linewidth=1, label='voltage RPM curve')
    
    # manufacturer data + regression curve for current
    ax1[i][1].plot(throttle, manufacturerCurrentCurve, color='black', linewidth=1, linestyle='dashed')#, label='rpm regression')
    ax1[i][1].scatter(manufacturerThrottle, manufacturerCurrent, color='black', marker='x', label='manufacturer current')
    
    # regression curve for ESC current
    ax1[i][1].plot(motor[m].throttleRange, motor[m].ESCcurrentRegressionCurve,
                   color='deepskyblue', linestyle='dashed', linewidth=1)
    
    # regression curve for multimeter current
    ax1[i][1].plot(motor[m].throttleRange, motor[m].multimeterCurrentRegressionCurve,
                   color='red', linestyle='dashed', linewidth=1)
    
    # labels, title, grid, legend
    ax1[i][0].set_xlabel('Power [%]')
    ax1[i][0].set_ylabel('$RPM$')
    ax1[i][0].set_ylim(0,5000)
    ax1[i][0].grid()
    ax1[i][0].legend()
    ax1[i][0].set_title(f'Motor {m+1}: RPM over time')
    
    ax1[i][1].set_xlabel('Power [%]')
    ax1[i][1].set_ylabel('Current')
    ax1[i][1].set_ylim(0,35)
    ax1[i][1].grid()
    ax1[i][1].legend()
    ax1[i][1].set_title(f'Motor {m+1}: Current over time')

fig1.tight_layout()